# Broadcasting <a class="anchor" id="broadcasting">

Broadcasting is used by <code>numpy</code> with elementwise computations between arrays whose shapes do not match.  When two such arrays are added, for example, <code>numpy</code> adapts the arrays by repeating some elements in such a way that the shapes of the two arrays do match.  We will describe this mechanism in a more detailed fashion later in this notebook.
    
Before embarking on more complex broadcasting schemes, we will introduce the concept with some easily understood examples as shown below.

![vector_plus_value](images/elewise_vec_val.jpg)
    
![array_plus_vector](images/array_plus_vec.jpg)

We sometimes need to add constants to every element of an <span style="font-family:'Courier New'">ndarray</span>.  We might, similarly, need to add the elements of a 1D <span style="font-family:'Courier New'">ndarray</span> (think vector) to every row or column of a 2D <span style="font-family:'Courier New'">ndarray</span> (think matrix).  <span style="font-family:'Courier New'">numpy</span> gives us the capability to do that easily without expanding the smaller <span style="font-family:'Courier New'">ndarray</span> into an array of the same size as the larger <span style="font-family:'Courier New'">ndarray</span>.

## Add constant to elements of 1D and 2D <span style="font-family:'Courier New'">ndarray</span>s <a class="anchor" id="add-cnst-to-1d-2d">

In [ ]:
import numpy as np

In [ ]:
vec = np.array(np.arange(10))
mat = np.array(np.arange(16)).reshape(4,4)
vec, mat

In [ ]:
vec = vec + 1
vec

In [ ]:
mat = mat + 1
mat

## Adding Arrays
In the first example, there is only one way to add the vector to the mattrix due to the dimensions on those entities.  There may be multiple ways that the addition could be done, and the default may not be what you intend to do.

In this first example, there is only one alternative that makes sense for addition.

In [ ]:
mat = np.array(np.arange(15)).reshape(5,3)
vec = np.arange(3)

print(mat)
print(vec)

In [ ]:
mat + vec

In this second example, with an array with equal number of rows and columns the addition could be done in two ways.  The default here is to add the 1D array to each row.

In [ ]:
mat = np.array(np.arange(16)).reshape(4,4)
vec = np.arange(4)

print(mat)
print(vec)

In [ ]:
mat + vec

The dimensions of <code>mat</code> and <code>vec</code> are such that <code>vec</code> could have been added to either each row or each column.  The default here is to add <code>vec</code> to each row.  We will explain later why this is the case.

If you want to add the 1D vector to the columns, then you can transform it into a column vector using either <code>np.newaxis()</code> or <code>reshape()</code>, or you may alternately view it as a 4 by 1 array, which causes the 1D array to be added to the columns.  The <code>-1</code> argument indicates that <code>numpy</code> should use whatever number of rows is appropriate given the number to total elements in the array.

In [ ]:
mat = np.array(np.arange(16)).reshape(4,4)
vec = np.arange(4)
#vec = vec.reshape(-1,1)
vec = vec[:,np.newaxis]

print(mat)
print(vec)

In [ ]:
np.add(mat, vec)

In [ ]:
mat + vec

In [ ]:
vec = np.arange(4)
vec[:,np.newaxis]

Alternately, the vector could be reshaped with the <code>.reshape()</code> method.

In [ ]:
vec = np.arange(4)
vec = vec.reshape(-1,1)

mat + vec

In [ ]:
vec = np.arange(4)
vec.reshape(-1,1)

A <code>-1</code> in a reshape statement tells <code>numpy</code> to figure out whatever dimension makes sense in that location based on the dimensions that the programmer provides and the total number of elements in the array.

## How Broadcasting Works.. in More Specific Terms

To predict how <code>numpy</code> will broadcast, that is, repeat elements from one array such that the dimensions of all arrays in a calculation will match, we need to compare the shape of arrays along all their dimensions.  We do this by writing the shapes of each dimension of an array from right to left in the table below.  Arrays with fewer dimensions will not have entries in some of the leftmost columns.  For example, take <code>mat</code> and <code>vec</code> from a previous example.

| Array|Shape|Shape|
|---|---|---|
|<code>mat</code>|4|4|
|<code>vec</code>| |4|

Where an array has fewer dimensions than another, <code>numpy</code> interprets the missing dimensions as having a shape of 1.

| Array|Shape|Shape|
|---|---|---|
|<code>mat</code>|4|4|
|<code>vec</code>|1|4|

Array dimensions where a first array has a shape alrger than one (<code>mat</code>) where a second array has an actually or implied dimension of 1 (<code>vec</code>), the second array will be repeated along the dimension where it has a shape of 1.  In this case, this explains why the elements of <code>vec</code> were repeated across the rows since the first column above describes rows.

In the two examples where we created a new axis in <code>vec</code>, our comparison table looks like this

| Array|Shape|Shape|
|---|---|---|
|<code>mat</code>|4|4|
|<code>vec</code>|4|1|

Now, <code>vec</code> elements will be repeated as indicated in the rightmost column, which is across the columns.

If two arrays have different shapes along any dimension and neither fo the shapes is one, then broadcasting will fail.

## A Purposeful Example

Suppose we have two 2D arrays whose rows represent <code>x-y</code> coordinates and we want to compute the difference of all pairs of <code>x-y</code> coordinates between the two tables.  We can accomplish that by taking advantage of broadcasting.

In [ ]:
a = np.random.randint(0,9,(4,2))
b = np.random.randint(0,9,(3,2))
a,b

First of all, we couldn't broadcast between these two arrays as is anyway because they ahve a different number of rows.

| Array|Shape|Shape|
|---|---|---|
|<code>a</code>|4|2|
|<code>b</code>|3|2|

If we insert another axis into <code>a</code> as shown in the next table,

| Array|Shape|Shape|Shape|
|---|---|---|---|
|<code>a</code>|4|1|2|
|<code>b</code>||3|2|

and so the 3 <code>x-y</code> coordinates (rows) in <code>b</code> are repeated along axis 1 via broadcasting.  This effectively permits us to compute the differences of all pairs.


In [ ]:
a[:,np.newaxis,:]  - b

In [ ]:
(a[:,np.newaxis,:]  - b).shape

In [ ]:
a[:,np.newaxis,:]

The result has shape 4 along <code>axis=2</code> reflecting the number of <code>x-y</code> coordinates in <code>a</code> and a shape of 3 along <code>axis=1</code> reflecting the number of <code>x-y</code> coordinates in <code>b</code>.

We can get a better view of how broacasting works by adding <code>b</code> to an array of zeros the same shape as <code>a[:,np.newaxis,:]</code> and, conversely, adding <code>a[:,np.newaxis,:]</code> to an array of zeros the same shape as <code>b</code>.

In [ ]:
np.zeros((a.shape[0],1,a.shape[1])) + b

In [ ]:
np.zeros(b.shape) + a[:,np.newaxis,:]

# <code>np.tile(np.array, reps)</code> 

This function creates a new array by repeating the element values of an original array, shown as <code>np.array</code> above. If <code>reps</code> is an integer, than the values of the original array are repeated across new columns.  If <code>reps</code> is a tuple, then new rows are added as well.  

In [ ]:
x = np.arange(3)
x

In [ ]:
np.tile(x,3)

In [ ]:
np.tile(x,(3,2))

Or, if you want to create some number of rows each of which is a replicate of the values from the original array, then you can use the following statement.

In [ ]:
np.tile(x,(3,1))

This method works also for replicating values from multi-dimensional arrays.

In [ ]:
y = np.arange(6).reshape(2,3)
y

In [ ]:
np.tile(y,2)

In [ ]:
np.tile(y, (2,2))

# <code>np.repeat()</code>

This method does an operation similar to <code>np.tile()</code>, except that each element value from an original array is repeated in a different mode than in <code>np.tile()</code>.  <code>np.tile()</code> replicates all the values from the original array before repeating values, but <code>np.repeat()</code> repeats each value individually before moving to the next value.  The result of using this method on multi-dimensional arrays will be a flattened array unless the <code>axis</code> argument is used.

In [ ]:
np.repeat(3,4)

In [ ]:
x = np.arange(6).reshape(2,3)
x

In [ ]:
np.repeat(x,3)

In [ ]:
np.repeat(x,3,axis=0)

In [ ]:
np.repeat(x,3,axis=1)